In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder.getOrCreate()

In [2]:
data = [('A', 3), ('A', 4), ('A', -2),
('B', -3), ('B', 4), ('B', 5),
('C', 4),
('D', -7), ('D', -9),
('E', -5),
('F', -2), ('F', 0)]

In [3]:
rdd= spark.sparkContext.parallelize(data, 3)

In [4]:
count_data=rdd.mapValues(lambda x:(x,1))

In [5]:
count_data.collect()

[('A', (3, 1)),
 ('A', (4, 1)),
 ('A', (-2, 1)),
 ('B', (-3, 1)),
 ('B', (4, 1)),
 ('B', (5, 1)),
 ('C', (4, 1)),
 ('D', (-7, 1)),
 ('D', (-9, 1)),
 ('E', (-5, 1)),
 ('F', (-2, 1)),
 ('F', (0, 1))]

## Mean with reducebykey

In [6]:
count_data.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).mapValues(lambda x: x[0]/x[1] ).collect()

[('B', 2.0),
 ('C', 4.0),
 ('A', 1.6666666666666667),
 ('F', -1.0),
 ('D', -8.0),
 ('E', -5.0)]

## Mean with groupbykey

In [8]:
rdd.groupByKey().mapValues(lambda x:sum(x)/len(x)).collect()

[('B', 2.0),
 ('C', 4.0),
 ('A', 1.6666666666666667),
 ('F', -1.0),
 ('D', -8.0),
 ('E', -5.0)]

In [9]:
rdd.collect()

[('A', 3),
 ('A', 4),
 ('A', -2),
 ('B', -3),
 ('B', 4),
 ('B', 5),
 ('C', 4),
 ('D', -7),
 ('D', -9),
 ('E', -5),
 ('F', -2),
 ('F', 0)]

In [18]:
sorted_rdd=rdd.sortBy(lambda x: x[1])

In [19]:
sorted_rdd.collect()

[('D', -9),
 ('D', -7),
 ('E', -5),
 ('B', -3),
 ('A', -2),
 ('F', -2),
 ('F', 0),
 ('A', 3),
 ('A', 4),
 ('B', 4),
 ('C', 4),
 ('B', 5)]

In [33]:
sorted_rdd.groupByKey().mapValues(lambda x:list(x)).collect()

[('B', [-3, 4, 5]),
 ('C', [4]),
 ('A', [-2, 3, 4]),
 ('F', [-2, 0]),
 ('D', [-9, -7]),
 ('E', [-5])]

## Median with groupbykey

In [41]:
sorted_rdd.groupByKey().mapValues(lambda x:list(x)).\
mapValues(lambda x:((x[int ((len(x)-1)/2)]+x[int (len(x)/2)])/2)).collect()

[('B', 4.0), ('C', 4.0), ('A', 3.0), ('F', -1.0), ('D', -8.0), ('E', -5.0)]

## Median with reducebykey

In [63]:
sorted_rdd.map(lambda x: (x[0], list(x[1:]))).reduceByKey(lambda x,y:x+y).\
mapValues(lambda x:((x[int ((len(x)-1)/2)]+x[int (len(x)/2)])/2)).collect()

[('B', 4.0), ('C', 4.0), ('A', 3.0), ('F', -1.0), ('D', -8.0), ('E', -5.0)]